In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from scipy import stats
from matplotlib import style
import seaborn as sns
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import graphviz as gr
from linearmodels.iv import IV2SLS

%matplotlib inline

pd.set_option("display.max_columns", 5)
style.use("fivethirtyeight")

In [2]:
data = pd.read_csv("../Casual-Inference/data/income_data/train.csv")
data = data.dropna()
data.describe()
data

,age,workclass,...,native-country,income_>50K
0,67,Private,...,United-States,1
1,17,Private,...,United-States,0
2,31,Private,...,United-States,1
3,58,State-gov,...,United-States,0
4,25,State-gov,...,United-States,0
...,...,...,...,...,...
43952,52,Private,...,United-States,1
43953,19,Private,...,United-States,0
43954,30,Private,...,United-States,0
43955,46,Private,...,United-States,0


In [3]:
data = data.loc[:, ["race", "educational-num", "age", "income_>50K", "occupation", "relationship", "marital-status", "native-country", "gender"]]
data.rename(columns = {'educational-num':'educational_num', "income_>50K": "income_bigger_than_50K", 'marital-status':'marital_status'}, inplace = True)
data["race"] = data["race"].replace(to_replace ="Amer-Indian-Eskimo",
                 value ="Indian")
data["race"] = data["race"].replace(to_replace ="Asian-Pac-Islander",
                 value ="Asian")
occupationDict = {
    "Exec-managerial" : 0,
    "Other-service" : 1,
    "Transport-moving" : 2,
    "Adm-clerical" : 3,
    "Machine-op-inspct" : 4,
    "Sales" : 5,
    "Handlers-cleaners" : 6,
    "Farming-fishing" : 7,
    "Protective-serv" : 8,
    "Prof-specialty" : 9,
    "Craft-repair" : 10,
    "Tech-support" : 11,
    "Priv-house-serv" : 12,
    "Armed-Forces" : 13,
    "" : -1
}
raceDict = {
    "White" : 0,
    "Black" : 1,
    "Asian" : 2,
    "Indian" : 3,
    "Other" : 4,
    "" : -1
}
genderDict = {
    "Male" : 0,
    "Female" : 1,
    "" : -1
}
def map_age(age):
    if age < 20 or age > 90: return -1
    if 20 <= age < 25: return 1
    if 25 <= age < 30: return 2
    if 30 <= age < 35: return 3
    if 35 <= age < 40: return 4
    if 40 <= age < 45: return 5
    if 45 <= age < 50: return 6
    if 50 <= age < 60: return 7
    if 60 <= age < 75: return 8
    if 75 <= age < 90: return 9

data["occupation"] = data["occupation"].map(occupationDict)
data["race"] = data["race"].map(raceDict)
data["gender"] = data["gender"].map(genderDict)
data["age_interval"] = data["age"].map(map_age)
data

,race,educational_num,...,gender,age_interval
0,0,16,...,0,8.0
1,0,8,...,0,-1.0
2,0,13,...,0,3.0
3,0,4,...,0,7.0
4,1,10,...,0,2.0
...,...,...,...,...,...
43952,0,13,...,0,7.0
43953,0,9,...,1,-1.0
43954,0,10,...,0,3.0
43955,0,13,...,1,6.0


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from tensorflow import keras

Using TensorFlow backend.


In [5]:
X = data[['educational_num','age']]
X

,educational_num,age
0,16,67
1,8,17
2,13,31
3,4,58
4,10,25
...,...,...
43952,13,52
43953,9,19
43954,10,30
43955,13,46


In [8]:
y = data['occupation']
y

0        0
1        1
2        0
3        2
4        1
        ..
43952    0
43953    1
43954    5
43955    5
43956    6
Name: occupation, Length: 40727, dtype: int64

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.15)

In [19]:
encoder=LabelEncoder()
y_train_encoded=encoder.fit_transform(y_train)
y_val_encoded=encoder.fit_transform(y_val)
y_test_encoded=encoder.fit_transform(y_test)

y_train_cat=to_categorical(y_train_encoded)
y_val_cat=to_categorical(y_val_encoded)
y_test_cat=to_categorical(y_test_encoded)


In [22]:
network = Sequential()
network.add(Dense(units=128, activation='relu'))
network.add(Dense(units=128, activation='relu'))
network.add(Dense(units=64, activation='relu'))
network.add(Dense(units=14, activation='softmax'))
opt=keras.optimizers.Adam(learning_rate=0.01,decay=0.0005)
network.compile(
  optimizer = opt,
  loss = 'categorical_crossentropy', 
  metrics = ['accuracy']
)

network.fit(X_train,y_train_cat,validation_data=(X_val,y_val_cat),epochs=200)
test_loss, test_acc = network.evaluate(X_test,y_test_cat)

print('The loss function on our test set: ' + str(test_loss))
print('Accuracy on our test set: ' + str(test_acc))

Epoch 1/200
974/974 [==============================] - 1s 1ms/step - loss: 2.2468 - accuracy: 0.2141 - val_loss: 2.2057 - val_accuracy: 0.2228
Epoch 2/200
974/974 [==============================] - 1s 1ms/step - loss: 2.1405 - accuracy: 0.2448 - val_loss: 2.1203 - val_accuracy: 0.2537
Epoch 3/200
974/974 [==============================] - 1s 1ms/step - loss: 2.1270 - accuracy: 0.2446 - val_loss: 2.1218 - val_accuracy: 0.2520
Epoch 4/200
974/974 [==============================] - 1s 919us/step - loss: 2.1210 - accuracy: 0.2479 - val_loss: 2.1120 - val_accuracy: 0.2526
Epoch 5/200
974/974 [==============================] - 1s 933us/step - loss: 2.1159 - accuracy: 0.2509 - val_loss: 2.1118 - val_accuracy: 0.2588
Epoch 6/200
974/974 [==============================] - 1s 898us/step - loss: 2.1131 - accuracy: 0.2493 - val_loss: 2.1078 - val_accuracy: 0.2539
Epoch 7/200
974/974 [==============================] - 1s 905us/step - loss: 2.1095 - accuracy: 0.2518 - val_loss: 2.1120 - val_accuracy